In [11]:
from models.SMPModel import SMPModel

model = SMPModel(encoder_name="resnet50",  n_channels=35, flatten_temporal_dimension=True,
                 pos_class_weight=962, encoder_weights="imagenet", loss_function="Dice")


from dataloader.FireSpreadDataset import FireSpreadDataset
from dataloader.FireSpreadDataModule import FireSpreadDataModule

datamodule = FireSpreadDataModule(
    n_leading_observations=5,  # Replace with actual values
    features_to_keep=[0, 1, 2, 3, 4, 38, 39],
    remove_duplicate_features=True,
    data_fold_id=2,
    data_dir="/home/sl221120/scratch/WildfireSpreadTS_HDF5",
    batch_size=64, 
    n_leading_observations_test_adjustment=5,
    crop_side_length=128, 
    load_from_hdf5=True, 
    num_workers=32
)

Loaded resnet50 with imagenet weights


In [14]:
ckpt = "./lightning_logs/wildfire_progression/w0k3m5oa/checkpoints/epoch=50-step=4947.ckpt"

/home/sl221120/WildfireSpreadTS/src/models/SwinUnetLightning.py

TypeError: object of type 'NoneType' has no len()

In [1]:
from pytorch_lightning.utilities import rank_zero_only
import torch
from dataloader.FireSpreadDataModule import FireSpreadDataModule
from pytorch_lightning.cli import LightningCLI
from models import SMPModel, BaseModel, ConvLSTMLightning, LogisticRegression  # noqa
from models import BaseModel
import wandb
import os
from dataloader.FireSpreadDataset import FireSpreadDataset
from dataloader.utils import get_means_stds_missing_values
from PIL import Image
import torch.nn.functional as F

/home/sl221120/anaconda3/envs/fire_ts/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import torch
import os
from pytorch_lightning import Trainer
from models import BaseModel
from dataloader import FireSpreadDataModule  
import wandb

# Step 1: Define the path to the checkpoint

ckpt_path = "/home/sl221120/WildfireSpreadTS/lightning_logs/wildfire_progression/w0k3m5oa/checkpoints/epoch=50-step=4947.ckpt"

# Step 2: Instantiate the model (Make sure to provide the right arguments)
# Here, we assume the checkpoint has the model's parameters saved. You may need to pass additional arguments.
model = BaseModel.load_from_checkpoint(ckpt_path)

# Step 3: Instantiate the datamodule
datamodule = FireSpreadDataModule(
    n_leading_observations=5,  # Replace with actual values
    features_to_keep=[0, 1, 2, 3, 4, 38, 39],
    remove_duplicate_features=True,
    data_fold_id=2  # or other required args
)

# Step 4: Set up the trainer
trainer = Trainer()

# Step 5: Use the trainer to predict
prediction_output = trainer.predict(model, datamodule)

# Step 6: Process predictions
# Extract input fire masks, predictions, and targets (same as in your CLI code)
x_af = torch.cat(
    list(map(lambda tup: tup[0][:, -1, :, :].squeeze(), prediction_output)), axis=0)
y = torch.cat(list(map(lambda tup: tup[1], prediction_output)), axis=0)
y_hat = torch.cat(list(map(lambda tup: tup[2], prediction_output)), axis=0)

fire_masks_combined = torch.cat([x_af.unsqueeze(0), y_hat.unsqueeze(0), y.unsqueeze(0)], axis=0)

# Step 7: Save the predictions to a file
predictions_file_name = os.path.join("/lightning_logs/wildfire_progression", f"predictions_best_swin_veg_multi.pt")
torch.save(fire_masks_combined, predictions_file_name)

print(f"Predictions saved to {predictions_file_name}")


RuntimeError: Error(s) in loading state_dict for BaseModel:
	Unexpected key(s) in state_dict: "model.swin_unet.patch_embed.proj.weight", "model.swin_unet.patch_embed.proj.bias", "model.swin_unet.patch_embed.norm.weight", "model.swin_unet.patch_embed.norm.bias", "model.swin_unet.layers.0.blocks.0.norm1.weight", "model.swin_unet.layers.0.blocks.0.norm1.bias", "model.swin_unet.layers.0.blocks.0.attn.relative_position_bias_table", "model.swin_unet.layers.0.blocks.0.attn.relative_position_index", "model.swin_unet.layers.0.blocks.0.attn.qkv.weight", "model.swin_unet.layers.0.blocks.0.attn.qkv.bias", "model.swin_unet.layers.0.blocks.0.attn.proj.weight", "model.swin_unet.layers.0.blocks.0.attn.proj.bias", "model.swin_unet.layers.0.blocks.0.norm2.weight", "model.swin_unet.layers.0.blocks.0.norm2.bias", "model.swin_unet.layers.0.blocks.0.mlp.fc1.weight", "model.swin_unet.layers.0.blocks.0.mlp.fc1.bias", "model.swin_unet.layers.0.blocks.0.mlp.fc2.weight", "model.swin_unet.layers.0.blocks.0.mlp.fc